In [1]:
# in this notebook we develop code for automatically binning for the unitary interpolation
# goal is to reduce the equation: f(N_1,...N_n) = 2^(n-1) * prod_i=1^n N_i + N_1 * prod_i=n (N_i + 1) + N_n * prod_i=1^(n-1) N_i
# where N_i are the bins in the directions i=1,...,n for the unitary interpolation

In [2]:
#### 
# generate latex table from arrays
from Analysis_Code.useful import *
import re

def tracelessify_AB(A,B):
    # Calculate the trace of A*B and A*A
    trace_AB = np.trace(np.dot(A, B))
    trace_AA = np.trace(np.dot(A, A))
    # If the trace of A*A is not zero, calculate the multiple c and create the new matrix C
    if np.abs(trace_AA) > 1e-10:  # We compare to a small number to avoid division by zero
        c = trace_AB / trace_AA
        C = B - c * A
    else:
        raise ValueError("The trace of A*A is zero, so we cannot create a new matrix C that fulfills Tr(AC) = 0.")
    return C

# A function that orthogonalizes the Hamiltonians in H_s, so that their products are traceless.UI_directional_overlap_traces
def orthogonalize_Hs(H_s):
    # use tracelessify_AB to orthogonalize the Hamiltonians in H_s for every combination of i and j
    n = H_s.shape[0]
    H_s_ortho = H_s.copy()
    for i in range(n):
        for j in range(i+1, n):
            H_s_ortho[j] = tracelessify_AB(H_s_ortho[i], H_s_ortho[j])
    return H_s_ortho

In [3]:
import numpy as np
from numpy.linalg import norm, eigh, qr
from scipy.linalg import schur

def Dag(U):
    return np.transpose(np.conj(U))

def vecxvec( E, wf, alpha): # Many times faster than multiply
    s = E.size
    for i in range(s):
        wf[i] = np.exp(-1j * E[i] * alpha) * wf[i]
    return wf
# Construct matrix exponentials and products faster than in numpy or scipy
def vm_exp_mul(E, V, dt = 1.0): # expm(diag(vector))*matrix  multiplication via exp(vector)*matrix
    s = E.size
    A = np.empty((s, s), np.complex128)
    for i in range(s):
        A[i,:] = np.exp(-1j * E[i] * dt) * Dag(V[:,i])
    return A
def expmH_from_Eig(E, V, dt = 1.0):
    U = np.dot(V, vm_exp_mul(E, V, dt))
    return U
def expmH(H, dt = 1.0):
    E, V = eigh(H)
    return expmH_from_Eig( E, V, dt)

##### Logarithms #########################################################
def unitary_eig(A): # alternative to np.eig returning unitary matrices V
    Emat, V = schur(A, output='complex')
    return np.diag(Emat), V
def vm_log_mul(E, V):
    s = E.size
    A = np.empty((s, s), np.complex128)
    for i in range(s):
        A[i,:] = np.log(E[i])*Dag(V[:,i])
    return A
def logmU(U):
    E, V = unitary_eig(U)
    return np.dot(V, vm_log_mul(E, V))

def logmU_parts(U):
    E, V = unitary_eig(U)
    return -np.log(E).imag, V

In [4]:
# make hamiltonian from weights c and H_s --> H = H_s[0]+ sum_i c[i]*H_s[i+1]
def H_from_c(H_s, c):
    H = H_s[0].copy()
    for i in range(len(c)):
        H = H + c[i]*H_s[i+1]
    return H

# Get unitary from weights c and H_s
def expm_H_s(H_s, c):
    H = H_from_c(H_s, c)
    E, V = eigh(H)
    return expmH_from_Eig( E, V )

# Construct the terms for corners of an interpolation (hyper-) cube
def make_border_unitaries(H_s, c_mins, c_maxs):
    if isinstance(c_mins, (list)):
        c_mins = np.array(c_mins, dtype=np.float64)
    if isinstance(c_maxs, (list)):
        c_maxs = np.array(c_maxs, dtype=np.float64)
    n = len(c_mins)
    s = H_s.shape[-1]
    U0 = expm_H_s(H_s, c_mins)
    Ui = np.empty((n, s, s), dtype=np.complex128)
    for i in range(n):
        curr_c = c_mins.copy()
        curr_c[i] = c_maxs[i]
        Ui[i] = expm_H_s(H_s, curr_c)
    n_ij = n * (n-1) // 2
    Uij_2 = np.empty((n_ij, s, s), dtype=np.complex128)
    Ui_2 = np.empty((n, s, s), dtype=np.complex128)
    ind = 0
    dc = (c_maxs - c_mins) 
    for i in range(n):
        curr_c = c_mins.copy()
        curr_c[i] += dc[i]/2
        Ui_2[i] = expm_H_s(H_s, curr_c)
        for j in range(i+1, n):
            curr_c = c_mins.copy()
            curr_c[i] += dc[i]/2
            curr_c[j] += dc[j]/2
            Uij_2[ind] = expm_H_s(H_s, curr_c)
            ind += 1
    curr_c = c_mins + dc/2
    U_2 = expm_H_s(H_s, curr_c)
    return U0, Ui, Ui_2, Uij_2, U_2

# Get the unitary interpolation along 
# Construct interpolation terms from U0 to Ui
def make_interpolation_terms(U0, Ui):
    n = Ui.shape[0]
    s = Ui.shape[-1]
    Es = np.empty((n,s), dtype=np.float64)
    Vs = np.empty((n,s,s), dtype=np.complex128)
    for i in range(n):
        U = np.dot(Ui[i], Dag(U0))
        E, V = logmU_parts(U)
        Es[i] = E
        Vs[i] = V
    return Es, Vs

def interpolation_core_term(Vs, Es, index, alpha):
    E = Es[index]
    V = Vs[index]
    return V @ np.diag(np.exp(-1j*E*alpha)) @ Dag(V)

def nd_interpolation_core_term(Vs, Es, U0, alphas, indices):
    U = interpolation_core_term(Vs, Es, indices[0], alphas[0])
    for i in range(1, len(indices)):
        U = np.dot(interpolation_core_term(Vs, Es, indices[i], alphas[i]), U)
    return U @ U0

In [5]:
# test directional accuracy of interpolation
def UI_directional_overlap_traces(U0, Ui_2, Uij_2, U_2, Es, Vs):
    # Test the quality of the interpolation for every combination (i,j) of directions at point x_i = 0.5 = x_j
    # exception 1d: test only center
    # Calculate and return the trace of the overlap operator M = U_exact^dagger @ U_interpolated (-d) , where d is the hilbert space dimension, which is subtracted except for all tr_M except the central element tr_M_2
    n = Es.shape[0]
    d = U0.shape[0] 
    if n == 1:
        alphas = np.array([0.5])
        indices = np.array([0])
        U = nd_interpolation_core_term(Vs, Es, U0, alphas, indices)
        tr_M = np.array([])
        tr_M_single = np.array([np.trace(Dag(Ui_2[0]) @ U) - d])
        tr_M_2 = tr_M_single[0]
        indexes = np.empty((0,0), dtype=np.int64)
    else:
        n_ij = n * (n-1) // 2
        tr_M = np.empty(n_ij, dtype=np.complex128)
        tr_M_single = np.empty(n, dtype=np.complex128)
        indexes = np.empty((n_ij, 2), dtype=np.int64)
        ind = 0
        alphas = np.array([0.5])
        for i in range(n):
            U = nd_interpolation_core_term(Vs, Es, U0, alphas, np.array([i]))
            tr_M_single[i] = np.trace(Dag(Ui_2[i]) @ U) - d
        alphas = np.array([0.5, 0.5])
        for i in range(n):
            for j in range(i+1, n):
                indices = np.array([i,j])
                U = nd_interpolation_core_term(Vs, Es, U0, alphas, indices)
                tr_M[ind] = np.trace(Dag(Uij_2[ind]) @ U) - d - tr_M_single[i] - tr_M_single[j]
                indexes[ind] = np.array([i,j], dtype=np.int64)
                ind += 1
    tr_M = tr_M + tr_M.conj()
    tr_M = tr_M.real.astype(np.float64)
    tr_M_single = tr_M_single + tr_M_single.conj()
    tr_M_single = tr_M_single.real.astype(np.float64)
    # compare central element
    if n > 2:
        alphas = np.array([0.5]*n)
        indices = np.arange(n)
        U = nd_interpolation_core_term(Vs, Es, U0, alphas, indices)
        tr_M_2 = np.trace(Dag(U_2) @ U) - d
        tr_M_2 = tr_M_2 + tr_M_2.conj()
        tr_M_2 = tr_M_2.real.astype(np.float64)
    elif n == 2:
        tr_M_2 = tr_M[0]
    else:
        tr_M_2 = tr_M_single[0]
    pref = -1/(d+1)
    tr_M = pref * tr_M
    tr_M_single = pref * tr_M_single
    tr_M_2 = pref * tr_M_2
    return tr_M_single, tr_M, tr_M_2, indexes

def get_bidirectional_overlap_operators(H_s, c_mins=None, c_maxs=None, bins=None): # Calculate the infidelity parameters for a given interpolation
    n = H_s.shape[0]-1
    if c_mins is None:
        c_mins = np.zeros(n)
    if c_maxs is None:
        c_maxs = np.ones(n)
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    else:
        bins = bins.astype(np.int64)
    dc = (c_maxs - c_mins)
    c_maxs = c_mins + dc / bins
    U0, Ui, Ui_2, Uij_2, U_2_exact = make_border_unitaries(H_s, c_mins, c_maxs)
    Es, Vs = make_interpolation_terms(U0, Ui)
    tr_M_single, tr_M, tr_M_2, indexes = UI_directional_overlap_traces(U0, Ui_2, Uij_2, U_2_exact, Es, Vs)  # , tr_M_2
    alphas = np.array([0.5]*n)
    indices = np.arange(n)
    U_2_approx = nd_interpolation_core_term(Vs, Es, U0, alphas, indices)
    return tr_M_single, tr_M, tr_M_2, indexes, U_2_exact, U_2_approx

def I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, param=None, bins=None):
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    if param is None:
        param = np.ones(n)
    param = param / bins
    res = 0.0
    if len(indexes) > 0:
        multipliers_2 = param[indexes[:,0]]**2*param[indexes[:,1]]**2
        res += np.sum(multipliers_2 * tr_M)
    multipliers_4 = param**4
    res +=  np.sum(multipliers_4 * tr_M_single)
    return res

def I_from_tr_M(tr_M_single, tr_M, indexes, n, param=None):
    return I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, param=param, bins=None)

In [6]:
# Check accuracy of the interpolation
import Analysis_Code.discrete_quantum as dq
import numpy as np
import matplotlib.pyplot as plt
printing = False
s_vals, n_vals, orth_vals, unorth_vals = [], [], [], []
rng = np.random.default_rng(100)
for n in range(1,5):
    for s in [4,16]:
        reps = 100
        rel_accs = np.empty(reps)
        n_vals.append(n)
        s_vals.append(s)
        for ortho in range(2):
            for r in range(reps):
                H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=np.pi/2*np.array([1.0,0.025]), rng=rng)
                if ortho:
                    H_s = orthogonalize_Hs(H_s) # orthogonalized matrices are no better than unorthogonalized ones
                tr_M_single, tr_M, tr_M_2, indexes, U_exact, U_approx = get_bidirectional_overlap_operators(H_s)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)

                I_exact = dq.Av_Infidelity(U_exact, U_approx)
                I_approx = I_from_tr_M(tr_M_single, tr_M, indexes, n)
                # relative accuracy
                rel_acc = np.abs(I_exact - I_approx) / I_exact
                rel_accs[r] = rel_acc
            if ortho:
                orth_vals.append([np.mean(rel_accs), np.std(rel_accs)])
                if printing:
                    print('Orthogonalized:   n=' + str(n) + ' s='+str(s)+' Relative accuracy: ', np.mean(rel_accs), '+/-', np.std(rel_accs))
            else:
                unorth_vals.append([np.mean(rel_accs), np.std(rel_accs)])
                if printing:
                    print('Unorthogonalized: n=' + str(n) + ' s='+str(s)+' Relative accuracy: ', np.mean(rel_accs), '+/-', np.std(rel_accs))

latex_str = latex_table(['n', 's', 'rel. acc.', 'rel. acc. (Orthogonalized)'], ['i', 'i', 'p2', 'p2'], [n_vals, s_vals, unorth_vals, orth_vals]) 
print(latex_str)

\begin{tabular}{|c|c|c|c|}\hline
n & s & rel. acc. & rel. acc. (Orthogonalized) \\
\hline
$1$ & $4$ & $(6.59\pm49.48)e-6$ & $(1.65\pm8.79)e-6$ \\
$1$ & $16$ & $(7.87\pm9.54)e-9$ & $(1.06\pm1.50)e-8$ \\
$2$ & $4$ & $(3.31\pm4.82)e-8$ & $(4.67\pm8.05)e-8$ \\
$2$ & $16$ & $(1.92\pm0.95)e-8$ & $(1.64\pm0.95)e-8$ \\
$3$ & $4$ & $(2.71\pm2.75)e-1$ & $(2.25\pm2.24)e-1$ \\
$3$ & $16$ & $(6.42\pm4.72)e-2$ & $(7.24\pm5.47)e-2$ \\
$4$ & $4$ & $(3.37\pm3.04)e-1$ & $(2.81\pm2.55)e-1$ \\
$4$ & $16$ & $(9.74\pm7.69)e-2$ & $(8.32\pm5.91)e-2$ \\
\hline
\end{tabular}


In [7]:
# estimate the infidelity for different binning and compare with actual accuracy
import Analysis_Code.discrete_quantum as dq
import numpy as np
import matplotlib.pyplot as plt
n = 4
s = 16  # Hilbert space dimension
bins = 32
param = np.ones(n)/bins   # scaled version
rng = np.random.default_rng(100)

rel_accs = np.empty(reps)
H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=np.pi/2*np.array([1.0,0.025]), rng=rng)
tr_M_single, tr_M, tr_M_2, indexes, U_exact, U_approx = get_bidirectional_overlap_operators(H_s)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)

I_approx = I_from_tr_M(tr_M_single, tr_M, indexes, n)
I_exact = dq.Av_Infidelity(U_exact, U_approx)
print('Original approximation: ', I_approx, 'Exact: ', I_exact)

I_approx2 = I_from_tr_M(tr_M_single, tr_M, indexes, n, param)
_, _, _, _, U_exact2, U_approx2 = get_bidirectional_overlap_operators(H_s, bins= bins * np.ones(n, dtype=np.int64))  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms
I_exact2 = dq.Av_Infidelity(U_exact2, U_approx2)
print('Extrapolated approximation: ', I_approx2, 'Exact: ', I_exact2)

print('Ratios: ',I_approx2/I_approx, ' (extrapolated) ', I_exact2/I_exact, ' (exact) ')

Original approximation:  2.747186064515758e-07 Exact:  2.984057987376332e-07
Extrapolated approximation:  2.619920792117842e-13 Exact:  2.8699265186560297e-13
Ratios:  9.5367431640625e-07  (extrapolated)  9.617529320130104e-07  (exact) 


In [102]:
import itertools
from scipy.optimize import minimize

def cache_size_by_bins(bins):
    # calculate the number of cache elements for a given binning
    n = bins.shape[0] #len(bins)
    if n > 1:
        n_C = 2**(n-1) * np.prod(bins)
        n_L = bins[0] * np.prod(bins[1:] + 1)
        n_R = bins[-1] * np.prod(bins[:-1] + 1) 
        N_tot = n_C + n_L + n_R
    else:
        N_tot = bins[0]
    return N_tot

def cache_size_change_by_dir(bins, negative=False):
    # check by how much the cache grows for every direction it could change in
    n = bins.shape[0] #len(bins)
    curr_size = cache_size_by_bins(bins)
    differences = np.zeros(len(bins))
    for i in range(n):
        curr_bins = bins.copy()
        if not negative:
            curr_bins[i] += 1
        else:
            curr_bins[i] -= 1
        new_size = cache_size_by_bins(curr_bins)
        differences[i] = new_size - curr_size
    return differences

def infidelity_change_by_dir(tr_M_single, tr_M, indexes, n, bins, negative=False):
    # check by how much the infidelity changes for every direction it could change in
    curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
    differences = np.zeros(len(bins))
    for i in range(n):
        curr_bins = bins.copy()
        if not negative:
            curr_bins[i] += 1
        else:
            curr_bins[i] -= 1
        new_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=curr_bins)
        differences[i] = curr_I - new_I 
    return differences

def _find_solution(tr_M_single, tr_M, indexes, n, bins, I_tar=10**-14):
    # increase to find solution
    curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
    while curr_I > I_tar:
        inf_diff = infidelity_change_by_dir(tr_M_single, tr_M, indexes, n, bins)
        cache_differences = cache_size_change_by_dir(bins)
        curr_prices_of_optimization = inf_diff / cache_differences # infidelity decrease per cache increase
        # any of the new points smaller than the target?
        new_Is = curr_I - inf_diff
        if np.any(new_Is < I_tar):
            # find the one with the smallest increase in cache size
            inds = np.where(new_Is < I_tar)
            min_index = np.argmin(cache_differences[inds])
            min_dir = inds[0][min_index]
            bins[min_dir] += 1
            curr_I = new_Is[min_dir]
        else:
            # find the one with the smallest cache price increase for the largest infidelity decrease
            max_ind = np.argmax(curr_prices_of_optimization)
            bins[max_ind] += 1
            curr_I = new_Is[max_ind]
    return bins, curr_I
def _optimize_solution(tr_M_single, tr_M, indexes, n, bins, I_tar=10**-14):
    # decrease for better solution
    optimizing = True # while the current binnning is changing keep optimizing= True
    curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
    curr_cache = cache_size_by_bins(bins)
    if curr_I > I_tar:
        raise Exception('Initial solution not < I_tar')
    new_Is = np.zeros(n)
    new_caches = np.zeros(n, dtype=np.int64)
    while optimizing:
        # check neighbors I
        for i in range(n):
            curr_bins = bins.copy()
            curr_bins[i] -= 1
            if curr_bins[i] < 1:
                new_Is[i] = 1.0
            else:
                new_Is[i] = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=curr_bins)
            # cache smaller?
            new_caches[i] = cache_size_by_bins(curr_bins)
        # find I < I_tar among now_Is
        inds = np.where(new_Is < I_tar)[0]
        if len(inds) > 0:
            min_index = np.argmin(new_caches[inds])
            potential_ind = inds[min_index]
            if new_caches[potential_ind] < curr_cache:
                bins[potential_ind] -= 1
                curr_I = new_Is[potential_ind]
                curr_cache = new_caches[potential_ind]
            else:
                optimizing = False
        else:
            optimizing = False
    return bins, curr_I

def optimize_infidelity_old(tr_M_single, tr_M, indexes, n, bins=None, I_tar=10**-14):
    # optimize binning to get minimum cache for target infidelity
    # while I > I_tar: add bins in a direction that increases infidelity the most per cache size, if more than one direction would achieve the target fidelity 
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    bins, curr_I = _find_solution(tr_M_single, tr_M, indexes, n, bins, I_tar)
    # check surrounding points for smaller cache size
    bins, curr_I = _optimize_solution(tr_M_single, tr_M, indexes, n, bins, I_tar)
    return bins, curr_I

#### new optimization routine #################################################################################################
# for every index i, separate the indexes and tr_M which contain i and those that dont
def separate_indexes(i, indexes, tr_M):
    # get indexes that contain i
    inds_i = np.where(indexes[:,0] == i)[0]
    inds_i = np.concatenate((inds_i, np.where(indexes[:,1] == i)[0]))
    # get indexes that dont contain i, the inds not in inds_i
    inds_not_i = np.setdiff1d(np.arange(len(indexes)), inds_i)
    indexes_not_i = indexes[inds_not_i]
    ind_i = indexes[inds_i]
    # reduce indexes_i to not contain i, make it a one d array
    indexes_i = np.empty(len(ind_i), dtype=np.int64)
    for j in range(len(ind_i)):
        indexes_i[j] = ind_i[j][0] if ind_i[j][0] != i else ind_i[j][1]
    tr_M_not_i = tr_M[inds_not_i]
    tr_M_i = tr_M[inds_i]
    return indexes_i, indexes_not_i, tr_M_i, tr_M_not_i

def separate_all_indexes(indexes, tr_M, n):	
    indexes_i = []
    indexes_not_i = []
    tr_M_i = []
    tr_M_not_i = []
    for i in range(n):
        inds_i, inds_not_i, tr_M_i_, tr_M_not_i_ = separate_indexes(i, indexes, tr_M)
        indexes_i.append(inds_i)
        indexes_not_i.append(inds_not_i)
        tr_M_i.append(tr_M_i_)
        tr_M_not_i.append(tr_M_not_i_)
    return indexes_i, indexes_not_i, tr_M_i, tr_M_not_i

def I_rest_from_tr_M_bins(i, tr_M_single, tr_M_not_i, indexes_not_i, n, bins):
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    param = 1 / bins
    res = 0.0
    if len(indexes_not_i) > 0:
        multipliers_2 = param[indexes_not_i[:,0]]**2*param[indexes_not_i[:,1]]**2
        res += np.sum(multipliers_2 * tr_M_not_i)
    multipliers_4 = param**4
    res +=  np.sum(multipliers_4[:i] * tr_M_single[:i]) + np.sum(multipliers_4[i+1:] * tr_M_single[i+1:])
    return res

def I_i_from_tr_M_bins(i, tr_M_single, tr_M_i, indexes_i, n, bins):
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    param = 1 / bins
    res = 0.0
    if len(indexes) > 0:
        multipliers_2 = param[indexes_i]**2
        res += np.sum(multipliers_2 * tr_M_i) * param[i]**2
    res +=  param[i]**4 * tr_M_single[i]
    return res

# optimize solution by reducing one bin direction by one and then finding the optimum along the other directions 
def optimum_along_single_direction(i, tr_M_single, tr_M_i, tr_M_not_i, indexes_i, indexes_not_i, n, bins, I_tar=10**-14):
    # rewrite as qudratic expression, to get optimum along one direction
    new_bins = bins.copy()
    success = False
    s = 1/bins**2
    Pii =tr_M_single[i]
    I = I_rest_from_tr_M_bins(i, tr_M_single, tr_M_not_i, indexes_not_i, n, bins) - I_tar
    if I < 0:
        Ci = np.sum(tr_M_i * s[indexes_i])
        # find optimum along s_i -> quadratic expression in s_i
        Ci_Pii = Ci/Pii/2
        sqrt_internal = Ci_Pii**2 - I/Pii
        if sqrt_internal > 0:
            sqrt_term = np.sqrt(sqrt_internal)
            s_i = - Ci_Pii - sqrt_term
            if s_i < 0:
                s_i = - Ci_Pii + sqrt_term
            # find the 
            success = True if s_i > 0 else False
            if success:
                # from s_i to_bins[i]
                bins_i = np.sqrt(1/s_i)
                new_bins = bins.copy()
                new_bins[i] = np.ceil(bins_i).astype(np.int64)
    return new_bins, success

def optimize_infidelity(tr_M_single, tr_M, indexes, n, bins=None, I_tar=10**-14):
    # optimize binning to get minimum cache for target infidelity
    # while I > I_tar: add bins in a direction that increases infidelity the most per cache size, if more than one direction would achieve the target fidelity 
    if bins is None:
        bins = np.ones(n, dtype=np.int64)
    bins, curr_I = _find_solution(tr_M_single, tr_M, indexes, n, bins, I_tar)
    # check surrounding points for smaller cache size
    bins, curr_I = _optimize_solution(tr_M_single, tr_M, indexes, n, bins, I_tar)

    # separate indexes and tr_M
    if len(indexes) > 0:
        indexes_i, indexes_not_i, tr_M_i, tr_M_not_i = separate_all_indexes(indexes, tr_M, n)
        # for every index i, reduce by one, check if solution is still in target range,
        # if not, find which directions first inner point has lower cache size. if non has lower cache size, stop and go back to previous solution
        last_bins = bins.copy()
        found_lower = True
        curr_min_cache = cache_size_by_bins(bins)
        while found_lower:
            found_lower = False
            for i in range(n):
                bin_red = last_bins.copy()
                if bin_red[i] > 1:
                    bin_red[i] -= 1
                    #I_bin_red = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bin_red)
                    #cache_bin_red = cache_size_from_bins(bin_red)
                    # check for every other direction, which point leads to I < I_tar, and save the cache needed
                    for j in range(n):
                        if j == i:
                            continue
                        else:
                            bin_j, success = optimum_along_single_direction(j, tr_M_single, tr_M_i[j], tr_M_not_i[j], indexes_i[j], indexes_not_i[j], n, bin_red, I_tar)
                            if success:
                                I_bin_j = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bin_j)
                                if not I_bin_j < I_tar:
                                    raise ValueError('I_bin_j='+str(I_bin_j)+' should be smaller than I_tar, algorithm is broken. (' + str(bin_red) + '->' + str(bin_j) + ')')
                                cache_bin_j = cache_size_by_bins(bin_j)
                                if cache_bin_j < curr_min_cache:
                                    curr_min_cache = cache_bin_j
                                    bins = bin_j
                                    found_lower = True
            if found_lower:
                last_bins = bins.copy()
        bins = last_bins
    return bins, curr_I

#optimal_bins, I_exp = optimize_infidelity_new(tr_M_single, tr_M, indexes, n, I_tar=I_tar)
#print(optimal_bins, I_exp)

In [103]:
import Analysis_Code.discrete_quantum as dq
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools

# Check the infidelity landscape as a function of binning
for n in [1,2,3,4]:
    print( ' ' )
    print('-'*50)
    print('Number of Control Parameters: ', n)
    print('-'*50)
    s = 16  # Hilbert space dimension
    max_bins = 50
    max_bins2 = 15
    max_bins_list = [max_bins] + [max_bins2]*(n-1)
    max_bins_range_list = [range(n) for n in max_bins_list]
    rng = np.random.default_rng(100)
    I_tar = 10**-12

    rel_accs = np.empty(reps)
    H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=np.pi/2*np.array([1.0,0.025, 0.01]), rng=rng)
    tr_M_single, tr_M, tr_M_2, indexes, U_exact, U_approx = get_bidirectional_overlap_operators(H_s)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)

    # now via optimization
    print('Optimal Binning via Optimization:')
    optimal_bins, I_exp = optimize_infidelity_old(tr_M_single, tr_M, indexes, n, I_tar=I_tar)
    print('Bins: ',optimal_bins, ('total: '+str(np.prod(optimal_bins))))
    print('Est. I: ', I_exp)
    _,_,_,_, U_exact, U_approx = get_bidirectional_overlap_operators(H_s, bins=optimal_bins)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)
    print('Actual I: ', dq.Av_Infidelity(U_exact, U_approx))
    print('-'*50)

    # now via new optimization
    print('Optimal Binning via New Optimization:')
    optimal_bins, I_exp = optimize_infidelity(tr_M_single, tr_M, indexes, n, I_tar=I_tar)
    print('Bins: ',optimal_bins, ('total: '+str(np.prod(optimal_bins))))
    print('Est. I: ', I_exp)
    _,_,_,_, U_exact, U_approx = get_bidirectional_overlap_operators(H_s, bins=optimal_bins)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)
    print('Actual I: ', dq.Av_Infidelity(U_exact, U_approx))
    print('-'*50)

    I_ = np.empty(max_bins_list, dtype=np.float64)
    costs = -np.ones(max_bins_list, dtype=np.float64)
    # Create iterator of all combinations of bins
    ijk = itertools.product(*max_bins_range_list)
    # iterate over all combinations of bins
    for ind in ijk: #tqdm(ijk, total=max_bins*max_bins2**(n-1)):
        bins = np.array(ind)+1
        curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
        I_[ind] = curr_I
        if curr_I < I_tar:
            costs[ind] = cache_size_by_bins(bins)

    print('='*50)
    # where is the minimum cost?
    min_cost_indexes = np.where(costs == np.min(costs[costs > 0]))
    optimal_bins = np.array([min_[0] for min_ in min_cost_indexes]).flatten() +1

    print('Optimal Binning via exhaustive Search:')
    print('Bins: ',optimal_bins, ('total: '+str(np.prod(optimal_bins))))
    print('Est. I: ', I_[min_cost_indexes][0])
    _,_,_,_, U_exact, U_approx = get_bidirectional_overlap_operators(H_s, bins=optimal_bins)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)
    print('Actual I: ', dq.Av_Infidelity(U_exact, U_approx))
    print('-'*50)


 
--------------------------------------------------
Number of Control Parameters:  1
--------------------------------------------------
Optimal Binning via Optimization:
Bins:  [11] total: 11
Est. I:  7.3744595224245e-13
Actual I:  7.377431998634165e-13
--------------------------------------------------
Optimal Binning via New Optimization:
Bins:  [11] total: 11
Est. I:  7.3744595224245e-13
Actual I:  7.377431998634165e-13
--------------------------------------------------
Optimal Binning via exhaustive Search:
Bins:  [11] total: 11
Est. I:  7.3744595224245e-13
Actual I:  7.377431998634165e-13
--------------------------------------------------
 
--------------------------------------------------
Number of Control Parameters:  2
--------------------------------------------------
Optimal Binning via Optimization:
Bins:  [17  7] total: 119
Est. I:  9.536336443187545e-13
Actual I:  9.670042544485113e-13
--------------------------------------------------
Optimal Binning via New Optimizatio

Optimal Binning via Optimization:
Bins:  [22  8  7] total: 1232
Est. I:  9.59940856963182e-13
Actual I:  1.1913803277252555e-12
--------------------------------------------------
Optimal Binning via New Optimization:
Bins:  [22  8  7] total: 1232
Est. I:  9.59940856963182e-13
Actual I:  1.1913803277252555e-12
--------------------------------------------------
Optimal Binning via exhaustive Search:
Bins:  [22  8  7] total: 1232
Est. I:  9.59940856963182e-13
Actual I:  1.1913803277252555e-12
--------------------------------------------------
 
--------------------------------------------------
Number of Control Parameters:  4
--------------------------------------------------
Optimal Binning via Optimization:
Bins:  [27 10  7  9] total: 17010
Est. I:  9.784056991349676e-13
Actual I:  1.0501599589929356e-12
--------------------------------------------------
Optimal Binning via New Optimization:
Bins:  [27 10  7  9] total: 17010
Est. I:  9.784056991349676e-13
Actual I:  1.0501599589929356e

In [3]:
## Compare how accurate the optimization is
# Check the infidelity landscape as a function of binning
n_param = np.array([1,2,3,4])
amps_lists = np.array([[1.0,0.025, 0.01], [1.0,0.025]], dtype=object)
search_lists = np.array([[32,10], [30,30]])
search_lists2 = np.array([[64,13], [40,40]])
reps = 100

filename = 'binning_optimization_results_n_'+str(n_param[0])+'_'+str(n_param[-1])+'.npz'
dir = os.path.join(os.getcwd(), 'Cache')
full_path = os.path.join(dir, filename)
redo = True
if os.path.exists(full_path):
    redo = False
    results = np.load(full_path, allow_pickle=True)
    # check if input parameters have changed (n_param, amps_lists, search_lists, search_lists2, reps)
    if not np.all(results['n_param'] == n_param):
        redo = True
    if not np.all(results['amps_lists'] == amps_lists):
        redo = True
    if not np.all(results['search_lists'] == search_lists):
        redo = True
    if not np.all(results['search_lists2'] == search_lists2):
        redo = True
    if not results['reps'] == reps:
        redo = True
    if redo == False:
        print('Loaded Results!')
        # unpack results
        exhaust_I_est = results['exhaust_I_est']
        exhaust_I_act = results['exhaust_I_act']
        exhaust_cache_size = results['exhaust_cache_size']
        optim_I_est = results['optim_I_est']
        optim_I_act = results['optim_I_act']
        optim_cache_size = results['optim_cache_size']
    else:
        print('Redoing Calculation!')
else:
    print('No Results Found!')
if redo:
    mat_size = (len(n_param), len(amps_lists), reps)
    # exhaustive search
    exhaust_I_est = np.empty(mat_size)
    exhaust_I_act = np.empty(mat_size)
    exhaust_cache_size = np.empty(mat_size, dtype=np.int64)
    # optimisation
    optim_I_est = np.empty(mat_size)
    optim_I_act = np.empty(mat_size)
    optim_cache_size = np.empty(mat_size, dtype=np.int64)
    rng = np.random.default_rng(100)
    for i, n in enumerate(n_param):
        print('-'*50)
        print('Number of Control Parameters: ', n)
        for j, (a, sl, sl2) in enumerate(zip(amps_lists, search_lists, search_lists2)):
            amps = np.pi/2*np.array(a)
            print('Amplitudes: ', amps)
            s = 16  # Hilbert space dimension 
            I_tar = 10**-12
            for rep in tqdm(range(reps)):
                rel_accs = np.empty(reps)
                H_s = dq.Random_parametric_Hamiltonian_gauss(n, s, sigmas=amps, rng=rng)
                tr_M_single, tr_M, tr_M_2, indexes, U_exact, U_approx = get_bidirectional_overlap_operators(H_s)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)

                # now via optimization
                optimal_bins, I_exp = optimize_infidelity(tr_M_single, tr_M, indexes, n, I_tar=I_tar)
                cache_size = np.prod(optimal_bins)
                _,_,_,_, U_exact, U_approx = get_bidirectional_overlap_operators(H_s, bins=optimal_bins)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)
                I_actual = dq.Av_Infidelity(U_exact, U_approx)
                optim_I_est[i,j,rep] = I_exp
                optim_I_act[i,j,rep] = I_actual
                optim_cache_size[i,j,rep] = cache_size

                max_bins = sl[0] #64
                max_bins2 = sl[1] #20
                max_bins_list = [max_bins] + [max_bins2]*(n-1)
                max_bins_range_list = [range(n) for n in max_bins_list]
                I_ = np.empty(max_bins_list, dtype=np.float64)
                costs = -np.ones(max_bins_list, dtype=np.float64)
                # Create iterator of all combinations of bins
                ijk = itertools.product(*max_bins_range_list)
                # iterate over all combinations of bins
                for ind in ijk: #tqdm(ijk, total=max_bins*max_bins2**(n-1)):
                    bins = np.array(ind)+1
                    curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
                    I_[ind] = curr_I
                    if curr_I < I_tar:
                        costs[ind] = cache_size_by_bins(bins)

                # where is the minimum cost?
                if np.any(costs > 0):
                    min_cost_indexes = np.where(costs == np.min(costs[costs > 0]))
                else:
                    max_bins = sl2[0] #64
                    max_bins2 = sl2[1] #20
                    max_bins_list = [max_bins] + [max_bins2]*(n-1)
                    max_bins_range_list = [range(n) for n in max_bins_list]
                    I_ = np.empty(max_bins_list, dtype=np.float64)
                    costs = -np.ones(max_bins_list, dtype=np.float64)
                    # Create iterator of all combinations of bins
                    ijk = itertools.product(*max_bins_range_list)
                    # iterate over all combinations of bins
                    for ind in ijk: #tqdm(ijk, total=max_bins*max_bins2**(n-1)):
                        bins = np.array(ind)+1
                        curr_I = I_from_tr_M_bins(tr_M_single, tr_M, indexes, n, bins=bins)
                        I_[ind] = curr_I
                        if curr_I < I_tar:
                            costs[ind] = cache_size_by_bins(bins)
                    min_cost_indexes = np.where(costs == np.min(costs[costs > 0]))
                    # combine min_cost indexes into one array
                    #min_cost_indexes = np.array([np.array(ind) for ind in min_cost_indexes]).flatten()
                    print('Larger search required...', min_cost_indexes)

                optimal_bins = np.array([min_[0] for min_ in min_cost_indexes]).flatten() +1

                cache_size = np.prod(optimal_bins)
                I_est = I_[min_cost_indexes][0]
                _,_,_,_, U_exact, U_approx = get_bidirectional_overlap_operators(H_s, bins=optimal_bins)  #tr_M_2 ## currently a sparse approach to infidelity calculation (only using the diagonal terms)
                I_act = dq.Av_Infidelity(U_exact, U_approx)
                exhaust_I_est[i,j,rep] = I_est
                exhaust_I_act[i,j,rep] = I_act
                exhaust_cache_size[i,j,rep] = cache_size
    if not os.path.exists(dir):
        os.makedirs(dir)
    np.savez(full_path, n_param=n_param, amps_lists=amps_lists, search_lists=search_lists, search_lists2=search_lists2, reps=reps, exhaust_I_est=exhaust_I_est, exhaust_I_act=exhaust_I_act, exhaust_cache_size=exhaust_cache_size, optim_I_est=optim_I_est, optim_I_act=optim_I_act, optim_cache_size=optim_cache_size)


Loaded Results!


In [15]:
# analyze how well the optimization did
# calculate the accuracy of the infidelity estimation
# for every value calculate mean and std
rel_I_est2act = np.abs((optim_I_act - optim_I_est)/optim_I_act)
rel_I_est_mean = np.mean(rel_I_est2act, axis=-1)
rel_I_est_std = np.std(rel_I_est2act, axis=-1)

rel_cache_est2act = np.abs((optim_cache_size - exhaust_cache_size)/exhaust_cache_size)
rel_cache_est_mean = np.mean(rel_cache_est2act, axis=-1)
rel_cache_est_std = np.std(rel_cache_est2act, axis=-1)
mean_cache = np.mean(optim_cache_size, axis=-1)
std_cache = np.std(optim_cache_size, axis=-1)

# Make table of results for latex
names = ['n', 'Control Amplitudes', 'Cache Size', 'Rel. Acc. Cache Optimum', 'Rel. Acc. Infidelity']
str_funs = ['i', 'le1', 'pm', 'p2', 'p2']
# create variables for table
n_par, amps, cache, rel_cache, rel_I = [], [], [], [], []
for i, n in enumerate(n_param):
    for j, a in enumerate(amps_lists):
        n_par.append(n)
        # reformat a
        a2 = [a_ for a_ in a[1:]]
        while len(a2) < n:
            a2.append(a2[-1])
        amps.append(a2)
        cache.append([mean_cache[i,j], std_cache[i,j]])
        rel_cache.append([rel_cache_est_mean[i,j], rel_cache_est_std[i,j]])
        rel_I.append([rel_I_est_mean[i,j], rel_I_est_std[i,j]])
arrays = [n_par, amps, cache, rel_cache, rel_I]
latex_str = latex_table(names, str_funs, arrays, vert_lines=False)
print(latex_str)

\begin{tabular}{c c c c c}\hline
n & Control Amplitudes & Cache Size & Rel. Acc. Cache Optimum & Rel. Acc. Infidelity \\
\hline
$1$ & $[2.5,\, 1]\num{e-2}$ & $9.7 \pm 1.10$ & $0 \pm 0$ & $(1.6 \pm 1.2)\num{e-3}$ \\
$1$ & $[2.5]\num{e-2}$ & $9.6 \pm 1.7$ & $0 \pm 0$ & $(1.7 \pm 1.3)\num{e-3}$ \\
$2$ & $[2.5,\, 1]\num{e-2}$ & $(1 \pm 0.3)e2$ & $(1.9 \pm 6.8)\num{e-3}$ & $(8.7 \pm 8)\num{e-2}$ \\
$2$ & $[2.5,\, 2.5]\num{e-2}$ & $(2.4 \pm 0.7)e2$ & $(9.3 \pm 44.1)\num{e-4}$ & $(2.10 \pm 2.6)\num{e-2}$ \\
$3$ & $[2.5,\, 1,\, 1]\num{e-2}$ & $(1.3 \pm 0.5)e3$ & $(4.6 \pm 12.3)\num{e-3}$ & $(8.9 \pm 6.5)\num{e-2}$ \\
$3$ & $[2.5,\, 2.5,\, 2.5]\num{e-2}$ & $(7.8 \pm 2.6)e3$ & $(1.1 \pm 3.7)\num{e-3}$ & $(7.4 \pm 5.4)\num{e-2}$ \\
$4$ & $[2.5,\, 1,\, 1,\, 1]\num{e-2}$ & $(2 \pm 0.9)e4$ & $(2.6 \pm 4.3)\num{e-2}$ & $(8.10 \pm 7.6)\num{e-2}$ \\
$4$ & $[2.5,\, 2.5,\, 2.5,\, 2.5]\num{e-2}$ & $(3.1 \pm 1.3)e5$ & $(5.8 \pm 12.4)\num{e-3}$ & $(9.4 \pm 6.4)\num{e-2}$ \\
\hline
\end{tabular}
